In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter

import matplotlib.pyplot as plt

import operator
from functools import reduce
from functools import partial

from timeit import default_timer
from utilities3 import *

from Adam import Adam

torch.manual_seed(0)
np.random.seed(0)

In [2]:
TRAIN_PATH = 'darcy/piececonst_r421_N1024_smooth1.mat'
TEST_PATH = 'darcy/piececonst_r421_N1024_smooth2.mat'

ntrain = 1000
ntest = 100

batch_size = 20
learning_rate = 0.001

epochs = 250
step_size = 100
gamma = 0.5

modes = 12
width = 32

r = 5
h = int(((421 - 1)/r) + 1)
s = h

In [3]:
reader = MatReader(TRAIN_PATH)
x_train = reader.read_field('coeff')[:ntrain,::r,::r][:,:s,:s]
y_train = reader.read_field('sol')[:ntrain,::r,::r][:,:s,:s]

reader.load_file(TEST_PATH)
x_test = reader.read_field('coeff')[:ntest,::r,::r][:,:s,:s]
y_test = reader.read_field('sol')[:ntest,::r,::r][:,:s,:s]

x_normalizer = UnitGaussianNormalizer(x_train)
x_train = x_normalizer.encode(x_train)
x_test = x_normalizer.encode(x_test)

y_normalizer = UnitGaussianNormalizer(y_train)
y_train = y_normalizer.encode(y_train)

x_train = x_train.reshape(ntrain,s,s,1)
x_test = x_test.reshape(ntest,s,s,1)

train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), batch_size=batch_size, shuffle=False)

In [4]:
class SpectralConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, modes1, modes2):
        super(SpectralConv2d, self).__init__()

        """
        2D Fourier layer. It does FFT, linear transform, and Inverse FFT.    
        """

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.modes1 = modes1 #Number of Fourier modes to multiply, at most floor(N/2) + 1
        self.modes2 = modes2

        self.scale = (1 / (in_channels * out_channels))
        self.weights1 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, self.modes1, self.modes2, dtype=torch.cfloat))
        self.weights2 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, self.modes1, self.modes2, dtype=torch.cfloat))

    # Complex multiplication
    def compl_mul2d(self, input, weights):
        # (batch, in_channel, x,y ), (in_channel, out_channel, x,y) -> (batch, out_channel, x,y)
        return torch.einsum("bixy,ioxy->boxy", input, weights)

    def forward(self, x):
        batchsize = x.shape[0]
        #Compute Fourier coeffcients up to factor of e^(- something constant)
        x_ft = torch.fft.rfft2(x)

        # Multiply relevant Fourier modes
        out_ft = torch.zeros(batchsize, self.out_channels,  x.size(-2), x.size(-1)//2 + 1, dtype=torch.cfloat, device=x.device)
        out_ft[:, :, :self.modes1, :self.modes2] = \
            self.compl_mul2d(x_ft[:, :, :self.modes1, :self.modes2], self.weights1)
        out_ft[:, :, -self.modes1:, :self.modes2] = \
            self.compl_mul2d(x_ft[:, :, -self.modes1:, :self.modes2], self.weights2)

        #Return to physical space
        x = torch.fft.irfft2(out_ft, s=(x.size(-2), x.size(-1)))
        return x

In [5]:
class FNO2d(nn.Module):
    def __init__(self, modes1, modes2,  width):
        super(FNO2d, self).__init__()

        """
        The overall network. It contains 4 layers of the Fourier layer.
        1. Lift the input to the desire channel dimension by self.fc0 .
        2. 4 layers of the integral operators u' = (W + K)(u).
            W defined by self.w; K defined by self.conv .
        3. Project from the channel space to the output space by self.fc1 and self.fc2 .
        
        input: the solution of the coefficient function and locations (a(x, y), x, y)
        input shape: (batchsize, x=s, y=s, c=3)
        output: the solution 
        output shape: (batchsize, x=s, y=s, c=1)
        """

        self.modes1 = modes1
        self.modes2 = modes2
        self.width = width
        self.padding = 9 # pad the domain if input is non-periodic
        self.fc0 = nn.Linear(3, self.width) # input channel is 3: (a(x, y), x, y)

        self.conv0 = SpectralConv2d(self.width, self.width, self.modes1, self.modes2)
        self.conv1 = SpectralConv2d(self.width, self.width, self.modes1, self.modes2)
        self.conv2 = SpectralConv2d(self.width, self.width, self.modes1, self.modes2)
        self.conv3 = SpectralConv2d(self.width, self.width, self.modes1, self.modes2)
        self.w0 = nn.Conv2d(self.width, self.width, 1)
        self.w1 = nn.Conv2d(self.width, self.width, 1)
        self.w2 = nn.Conv2d(self.width, self.width, 1)
        self.w3 = nn.Conv2d(self.width, self.width, 1)

        self.fc1 = nn.Linear(self.width, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        intermediate_stats = []
        grid = self.get_grid(x.shape, x.device)
        x = torch.cat((x, grid), dim=-1)
        x = self.fc0(x)
        x = x.permute(0, 3, 1, 2)
        x = F.pad(x, [0,self.padding, 0,self.padding])

        x_prev = x[..., :-self.padding, :-self.padding].detach().clone()
        x1 = self.conv0(x)
        x2 = self.w0(x)
        x = x1 + x2
        x = F.gelu(x)
        x_curr = x[..., :-self.padding, :-self.padding]

        if not self.training:
            residual = x_curr - x_prev
            intermediate_stats.append({
                'layer': 0,
                'residual_norm': residual.norm(dim=1).mean().item(),
                'relative_change': (residual.norm(dim=1) / (x_prev.norm(dim=1) + 1e-8)).mean().item(),
                'feature_norm': x_curr.norm(dim=1).mean().item()
            })

        x_prev = x[..., :-self.padding, :-self.padding].detach().clone()
        x1 = self.conv1(x)
        x2 = self.w1(x)
        x = x1 + x2
        x = F.gelu(x)
        x_curr = x[..., :-self.padding, :-self.padding]
        
        if not self.training:
            residual = x_curr - x_prev
            intermediate_stats.append({
            'layer': 1,
            'residual_norm': residual.norm(dim=1).mean().item(),
            'relative_change': (residual.norm(dim=1) / (x_prev.norm(dim=1) + 1e-8)).mean().item(),
            'feature_norm': x_curr.norm(dim=1).mean().item()
            })

        x_prev = x[..., :-self.padding, :-self.padding].detach().clone()
        x1 = self.conv2(x)
        x2 = self.w2(x)
        x = x1 + x2
        x = F.gelu(x)
        x_curr = x[..., :-self.padding, :-self.padding]
        
        if not self.training:
            residual = x_curr - x_prev
            intermediate_stats.append({
            'layer': 2,
            'residual_norm': residual.norm(dim=1).mean().item(),
            'relative_change': (residual.norm(dim=1) / (x_prev.norm(dim=1) + 1e-8)).mean().item(),
            'feature_norm': x_curr.norm(dim=1).mean().item()
            })

        x1 = self.conv3(x)
        x2 = self.w3(x)
        x = x1 + x2

        x = x[..., :-self.padding, :-self.padding]
        x = x.permute(0, 2, 3, 1)
        x = self.fc1(x)
        x = F.gelu(x)
        x = self.fc2(x)

        return x, intermediate_stats
    
    def get_grid(self, shape, device):
        batchsize, size_x, size_y = shape[0], shape[1], shape[2]
        gridx = torch.tensor(np.linspace(0, 1, size_x), dtype=torch.float)
        gridx = gridx.reshape(1, size_x, 1, 1).repeat([batchsize, 1, size_y, 1])
        gridy = torch.tensor(np.linspace(0, 1, size_y), dtype=torch.float)
        gridy = gridy.reshape(1, 1, size_y, 1).repeat([batchsize, size_x, 1, 1])
        return torch.cat((gridx, gridy), dim=-1).to(device)

In [6]:
# ...existing code...
model = FNO2d(modes, modes, width).cuda()
print(count_params(model))

optimizer = Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

myloss = LpLoss(size_average=False)
y_normalizer.cuda()

for ep in range(epochs):
    model.train()
    t1 = default_timer()
    train_l2 = 0
    for x, y in train_loader:
        x, y = x.cuda(), y.cuda()

        optimizer.zero_grad()
        out, _ = model(x)
        out = out.reshape(batch_size, s, s)
        out = y_normalizer.decode(out)
        y = y_normalizer.decode(y)

        loss = myloss(out.view(batch_size,-1), y.view(batch_size,-1))
        loss.backward()

        optimizer.step()
        train_l2 += loss.item()

    scheduler.step()

    model.eval()
    test_l2 = 0.0

    # accumulator for intermediate stats across test batches
    stats_acc = {}  # layer -> {'residual_norm': [...], 'relative_change': [...], 'feature_norm': [...]}

    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.cuda(), y.cuda()
            out, intermediate_stats = model(x)
            out = out.reshape(out.shape[0], s, s)  # use actual batch size
            out = y_normalizer.decode(out)

            test_l2 += myloss(out.view(out.shape[0],-1), y.view(out.shape[0],-1)).item()

            # collect intermediate stats (each entry already averaged over batch/spatial dims in forward)
            for st in intermediate_stats:
                layer = st['layer']
                stats_acc.setdefault(layer, {'residual_norm': [], 'relative_change': [], 'feature_norm': []})
                stats_acc[layer]['residual_norm'].append(st['residual_norm'])
                stats_acc[layer]['relative_change'].append(st['relative_change'])
                stats_acc[layer]['feature_norm'].append(st['feature_norm'])

    train_l2 /= ntrain
    test_l2 /= ntest

    t2 = default_timer()
    print(f'Epoch {ep}: Time {t2-t1:.2f}s, Train L2 Loss: {train_l2:.4f}, Test L2 Loss: {test_l2:.4f}')

    # print batch-averaged intermediate stats per layer
    if stats_acc:
        for layer in sorted(stats_acc.keys()):
            avg_res = np.mean(stats_acc[layer]['residual_norm'])
            avg_rel = np.mean(stats_acc[layer]['relative_change'])
            avg_feat = np.mean(stats_acc[layer]['feature_norm'])
            print(f'  Layer {layer}: avg_residual_norm={avg_res:.6f}, avg_relative_change={avg_rel:.6f}, avg_feature_norm={avg_feat:.6f}')
# ...existing code...

/home/pinakinchoudhary/dev/myenv/lib/python3.10/site-packages/torch/cuda/__init__.py:829: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


1188353
Epoch 0: Time 3.82s, Train L2 Loss: 0.1623, Test L2 Loss: 0.1143
  Layer 0: avg_residual_norm=4.572046, avg_relative_change=1.362711, avg_feature_norm=2.576057
  Layer 1: avg_residual_norm=2.769637, avg_relative_change=1.070381, avg_feature_norm=1.531293
  Layer 2: avg_residual_norm=1.921100, avg_relative_change=1.241828, avg_feature_norm=1.264718
Epoch 1: Time 3.38s, Train L2 Loss: 0.1023, Test L2 Loss: 0.0939
  Layer 0: avg_residual_norm=4.629599, avg_relative_change=1.370985, avg_feature_norm=2.668606
  Layer 1: avg_residual_norm=2.888058, avg_relative_change=1.079594, avg_feature_norm=1.627661
  Layer 2: avg_residual_norm=2.016628, avg_relative_change=1.253785, avg_feature_norm=1.278283
Epoch 2: Time 3.39s, Train L2 Loss: 0.0734, Test L2 Loss: 0.0612
  Layer 0: avg_residual_norm=4.704455, avg_relative_change=1.371740, avg_feature_norm=2.712328
  Layer 1: avg_residual_norm=2.956388, avg_relative_change=1.090449, avg_feature_norm=1.827752
  Layer 2: avg_residual_norm=2.127992